In [1]:
from theano.sandbox import cuda

In [22]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from keras.layers import Embedding

In [4]:
#path = "data/ml-20m/"
path = "/mnt/data/ml-latest-small/"
model_path = '/mnt/models/ml-latest-small/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

## Set up data

We're working with the movielens data, which contains one rating per row, like this:

In [46]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [45]:
movie_names = pd.read_csv(path + 'movies.csv')
movie_names.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [48]:
movies = ratings.movieId.unique()
users = ratings.userId.unique()
movie2idx = {movie_id:idx for idx, movie_id in enumerate(movies)}

In [ ]:
# movie2idx[ratings.movieId.max()]

In [25]:
n_movies = movies.movieId.nunique()
n_users = ratings.userId.nunique()

## Fit Dot Product Model

In [ ]:
movie_emb = Embedding(, )